In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interactive, widgets, HBox, VBox

csv_file_path = r'C:\Users\Sun Yixin\PycharmProjects\PythonProjectQ5\incidents_clean.csv'

df = pd.read_csv(csv_file_path, parse_dates=['starttime_new'])
df = pd.DataFrame(df)
df

,type,starttime_new,endtime_new,vild_primair_wegnummer,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,duration
0,vehicle_obstruction,2019-08-28 12:11:32,2019-12-11 11:32:28,A1,4.974663,52.346931,151160.933333
1,vehicle_obstruction,2019-08-28 12:11:32,2019-12-11 11:32:28,A9,4.716725,52.514820,151160.933333
2,vehicle_obstruction,2019-08-28 12:11:32,2019-12-11 11:32:28,A9,4.738364,52.609730,151160.933333
3,vehicle_obstruction,2019-08-28 12:11:32,2019-12-11 11:32:28,A35,6.824692,52.204929,151160.933333
4,vehicle_obstruction,2019-08-28 12:11:32,2019-12-11 11:32:28,A4,4.346407,52.041920,151160.933333
...,...,...,...,...,...,...,...
88846,vehicle_obstruction,2019-12-31 23:43:49,2019-12-31 23:43:49,NaN,4.475721,51.893230,0.000000
88847,accident,2019-12-31 23:46:10,2019-12-31 23:46:10,N246,4.701406,52.443363,0.000000
88848,vehicle_obstruction,2019-12-31 23:47:01,2019-12-31 23:47:01,N31,5.922628,53.165226,0.000000
88849,vehicle_obstruction,2019-12-31 23:55:23,2019-12-31 23:55:23,A4,4.313686,51.442677,0.000000


In [2]:
from ipywidgets import interactive, widgets, HBox, VBox
import matplotlib.pyplot as plt
import pandas as pd

# 使用您之前的数据加载代码来获取df
# df = pd.read_csv('your_path.csv')
df = pd.read_csv(csv_file_path, parse_dates=['starttime_new'])
df['hour'] = df['starttime_new'].dt.hour
df['day_of_week'] = df['starttime_new'].dt.dayofweek
# 映射每种事故类型到一个颜色
color_map = {
    'vehicle_obstruction': 'royalblue',
    'accident': 'darkcyan',
    'general_obstruction': 'orange'
}

type_list = ['vehicle_obstruction', 'accident', 'general_obstruction']

incident_dropdown = widgets.Dropdown(
    options=['all', 'vehicle_obstruction', 'accident', 'general_obstruction'],
    value='all',
    description='Incident Type:'
)

weekday_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=4,
    step=1,
    description='Weekday:',
    continuous_update=False
)

show_percentage_checkbox = widgets.Checkbox(
    value=False,
    description='Show Percentage',
    disabled=False
)

def plot_chart(incident_type, weekday, show_percentage):
    plt.figure(figsize=(14,7))
    bottom = [0] * 24

    if incident_type == "all":
        for t in type_list:
            filtered_df = df[(df['type'] == t) & (df['day_of_week'] == weekday)]
            counts = filtered_df.groupby('hour').size()
            plt.bar(counts.index, counts.values, bottom=bottom, label=t, color=color_map[t])
            bottom += counts.values

            if show_percentage:
                total_counts = df[df['day_of_week'] == weekday].groupby('hour').size()
                percentage = (counts / total_counts) * 100
                for i, (hour, value) in enumerate(percentage.items()):
                    plt.text(hour, bottom[i]-counts[i]/2, f"{value:.2f}%", ha='center', va='center', color='black', fontsize=8)
            
        plt.legend(title="Incident Type")
    else:
        filtered_df = df[(df['type'] == incident_type) & (df['day_of_week'] == weekday)]
        avg_duration = filtered_df.groupby('hour')['duration'].mean()
        avg_duration.plot(kind='bar', color=color_map[incident_type])

    plt.ylabel('Average Duration (min)')
    plt.xlabel('Hour of the Day')
    title = f'Average Duration of {incident_type} Incidents on {list("Mon Tue Wed Thu Fri")[weekday]} by Hour'
    if incident_type == "all":
        title = f'Average Duration of All Incidents on {list("Mon Tue Wed Thu Fri")[weekday]} by Hour'
    plt.title(title)
    plt.show()

interactive_plot = interactive(plot_chart, 
                               incident_type=incident_dropdown, 
                               weekday=weekday_slider,
                               show_percentage=show_percentage_checkbox)

output = interactive_plot.children[-1]
output.layout.height = '500px'
VBox([HBox(interactive_plot.children[:-1]), output])